## Phython-Mysql Menu driven connector program

### Problem Statement

At ABC supermarket store selling five items – Milo, Sugar, Lipton Tea, Bournvita, and Milk in packets, the quantity in store for each item is monitored. When the quantity in store is 5 (the Reorder level), new order is placed for a quantity of say, 20 (Re-order quantity) to replenish the stock for the item in question.   The sales policy is that if at least five packets of any of these products are bought, a five percent discount is given on the total payable by customers on the product. In any case, unit prices are different for all the products. 

### Task

Implement a menu-driven program in any language of your choice using a database backend as input and all outputs directed to a file for the following tasks:

(i)	Create a database for the ABC Supermarket containing Customer, Sales, Product and Inventory Table.

(ii)	Generate report on Quantity in stock, Quantity Sold To Date, Quantity Remaining in Stock, Needs Replenishment?, etc for each of the items being sold.


### Importing libraries

In [2]:
#This library is used to connect sql and python
import mysql.connector
#This is used to control the error that might occur during the connection
from mysql.connector import Error
#This library is used to create a dataframe 
import pandas as pd

### Necessary Functions

#### 1. General Purpose Functions

In [3]:
### 1. Python-Mysql Database Connector Function
def DatabaseConnector(user_name,host_name, password, database):
    """
    Docstrings:
    
    This function is used to connect to a Mysql database from python.
    Parameters
    ----------
    user_name : The username of the person who wants to access info from the database
    host_name : This is the host name of the database location. hist_name = localhost if database is located on the same computer
    password : The password of the person who wants to access info from the database
    database : This is the name of the database to be connected to.
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = password,
            database = database
        )
        print('Mysql Database connected successfully')
    except Error as err:
        print(f'Error: {err}\n')
    
    return connection

## 2. Mysql Query Executor function
def ExecuteQuery(connection, query):
    
    """
    Docstrings:
    
    This function is used to execute any sql query on Mysql database from python.
    
    Parameters
    ----------
    connection : This is the database to be connected to
    query : This is the sql query needed to be executed
    """
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
    except Error as err:
        print('Error : {err}')
    return result

#### 2. Specific Purpose Functions

In [14]:
### 1. This function access the quantity of the individual product in stock from the ABC database
def QuantityInStock(connection):
    get_quantity_in_store = """
    SELECT i.ProductID,  p.ProductName, i.InventoryOnHand as Quantity_in_Stock
    FROM Inventory as i INNER JOIN Product as p
    ON i.ProductID = p.ProductID;
    """
    QIS = ExecuteQuery(connection, get_quantity_in_store)#QIS : Quantity in Stock
    QIS = pd.DataFrame(QIS, columns = ['Product ID', 'Product Name', 'Quantity in stock']).set_index('Product ID')
    
    return QIS

### 2. This function access the quantity of the individual product sold to date from the ABC database

def QuantitySoldToDate(connection):
    get_QuantitySoldToDate = """
    SELECT i.ProductID,  p.ProductName, (i.InventoryReceived + i.StartingInventory - i.InventoryOnHand) as Quantity_sold_to_Date
    FROM Inventory as i INNER JOIN Product as p
    ON i.ProductID = p.ProductID;
    """
    QSD = ExecuteQuery(connection,get_QuantitySoldToDate) #QSD : Quantity Solld to Date
    QSD = pd.DataFrame(QSD, columns = ['Product ID', 'Product Name', 'Quantity sold to Date']).set_index('Product ID')
    
    return QSD

### 3. This function access the if there is need for replenishing the individual products in the ABC database
## Note : When the quantity in store is 5 (the Reorder level), new order is placed for a quantity of say, 20 (Re-order quantity) to replenish the stock for the item in question.

def NeedsForReplenishment(connection):
    NFR = QuantityInStock(connection = connection) #NFR : Needs for Replenishment
    NFR['Needs for Replenishment'] = NFR['Quantity in stock'].apply(lambda x: 'Yes' if x<=5 else 'No' )
    return NFR


## 4. This function creates a file for any of the above report upon request

def FileReport(df, filename, abbrev):
    with open(filename+'.txt', 'w') as file:
        file.write(f'                    {abbrev} Report\n')
        file.write('                    ----------------------------\n\n')
        for i in range(len(df)):
            index = df.index[i]
            p_name , p_quan = df.loc[index]['Product Name'],  df.loc[index][abbrev]
            file.write(f'Product ID: {index}, Product Name: {p_name}, {abbrev}: {p_quan}\n\n')
        print('Loading..........................')
        print('File created successfully!')
        
## 5. This function takes care of any instance of incorrect username or password

def LogInCheck(connect):
    if connect == None:
        print('Check the password and username to make sure they are correct\n')
        print('Press 1 to Continue\nPress other key to Quit Program')
        res = input('Input response : ')
        if res == '1':
            DatabaseInfoAccess()
        else:
            sys.exit()
    else:
        pass

In [15]:
import sys
import warnings
warnings.filterwarnings('ignore')

### 3. The Menu-Driven Function

This function generates report on Quantity in stock, Quantity Sold To Date, and Needs Replenishment for each of the items being sold by ABC Supermarket.

In [16]:
def DatabaseInfoAccess():
    print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< WELCOME >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')
    print('To access information from ABC Supermarket Database kindly provide your login details\n')
    user_name = input('Input Database Username : ')
    password = input('Input Database Password : ')
    connection = DatabaseConnector(user_name = user_name, host_name = 'localhost',
                                  password = password, database = 'ABC_Supermarket_Database')
    LogInCheck(connect = connection)
    def MainMenu():
        while(connection != None):
            print('\n')
            print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MAIN MENU >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')
            print('To access the following command select the corresponding number\n')
            print('1. Quantity of Product in stock')
            print('2. Quantity Sold To Date')
            print('3. Needs for Replenishment')
            print('4. Logout')
            print('5. Quit Program\n')
            response = input('Input response : ')

            if response == '1':
                filename = input('Input the name of the file: ')
                df = QuantityInStock(connection = connection)
                FileReport(df = df, filename=filename, abbrev = 'Quantity in stock')
                MainMenu()
                break
            elif response == '2':
                filename = input('Input the name of the file: ')
                df = QuantitySoldToDate(connection = connection)
                FileReport(df = df, filename=filename, abbrev = 'Quantity sold to Date')
                MainMenu()
                break
            elif response == '3':
                filename = input('Input the name of the file: ')
                df = NeedsForReplenishment(connection = connection)
                FileReport(df = df, filename=filename, abbrev = 'Needs for Replenishment')
                MainMenu()
                break
            elif response == '4': 
                DatabaseInfoAccess()
                break
            elif response == '5':
                break
            else:
                print('Invalid response\nInput a number between 1 and 5')
                MainMenu()
                break
    MainMenu()
                
                

### Testing the program

In [16]:
DatabaseInfoAccess()

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< WELCOME >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

To access information from ABC Supermarket Database kindly provide your login details

Input Database Username : root
Input Database Password : Babawale@208179
Mysql Database connected successfully


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MAIN MENU >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

To access the following command select the corresponding number

1. Quantity of Product in stock
2. Quantity Sold To Date
3. Needs for Replenishment
4. Logout
5. Quit Program

Input response : 1
Input the name of the file: Quantity_in_stock
Loading..........................
File created successfully!


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MAIN MENU >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

To access the following command select the corresponding number

1. Quantity of Product in stock
2. Quantity Sold To Date
3. Needs for Replenishment
4. Logout
5. Quit Program

Input response : 5


#### File report

In [24]:
with open('Quantity_in_stock.txt','r') as file:
    content = file.read()
    print(content)

                    Quantity in stock Report
                    ----------------------------

Product ID: ITM001, Product Name: Milo, Quantity in stock: 56

Product ID: ITM002, Product Name: Sugar, Quantity in stock: 96

Product ID: ITM003, Product Name: Lipton Tea, Quantity in stock: 13

Product ID: ITM004, Product Name: Bournvita, Quantity in stock: 34

Product ID: ITM005, Product Name: Milk, Quantity in stock: 20


